<a href="https://colab.research.google.com/github/MahimaMaurya/Indigo/blob/main/Notebooks/2_BERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForQuestionAnswering
from torch.utils.data import Dataset, DataLoader
import torch
import logging
import warnings

In [10]:

# Load the cleaned dataset
df = pd.read_csv('/content/tokenized_output.csv', on_bad_lines='skip')
df1 = pd.read_csv('/content/test_data.csv', on_bad_lines='skip')
df.head()

,question,tokenized_question,stemmed_question,answer,tokenized_answer,stemmed_answer,question_length,answer_length
0,How can a piece of paper folded 103 times be l...,"['how', 'can', 'a', 'piece', 'of', 'paper', 'f...","['how', 'can', 'a', 'piec', 'of', 'paper', 'fo...",It isn’t. It just has a stack height longer t...,"['it', 'isn’t', 'it', 'just', 'has', 'a', 'sta...","['it', 'isn’t', 'it', 'just', 'ha', 'a', 'stac...",81,119
1,Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",72,258
2,"If the Von Karman line is 62 miles above, then...","['if', 'the', 'von', 'karman', 'line', 'is', '...","['if', 'the', 'von', 'karman', 'line', 'is', '...",No. The Karman line has nothing to do with the...,"['no', 'the', 'karman', 'line', 'has', 'nothin...","['no', 'the', 'karman', 'line', 'ha', 'noth', ...",114,759
3,Do soldiers pick up fallen enemy weapons to us...,"['do', 'soldiers', 'pick', 'up', 'fallen', 'en...","['do', 'soldier', 'pick', 'up', 'fallen', 'ene...","It’s against the rules of war so no, if they r...","['it’s', 'against', 'the', 'rules', 'of', 'war...","['it’', 'against', 'the', 'rule', 'of', 'war',...",89,379
4,How do I get my son to stop making music?,"['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","Wha- if he loves making music, let him make mu...","['wha', 'if', 'he', 'loves', 'making', 'music'...","['wha', 'if', 'he', 'love', 'make', 'music', '...",41,244


In [11]:
df['question'] = df['question'].fillna('')
df['answer'] = df['answer'].fillna('')

In [12]:

logging.disable(logging.WARNING)
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some toke")
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

In [21]:
class BERTDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings)


In [23]:
from transformers import get_linear_schedule_with_warmup, AdamW
from tqdm import tqdm

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(df['question'].tolist(), df['answer'].tolist(), truncation=True, padding=True)


dataset = BERTDataset(train_encodings)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# Training parameters
epochs = 2
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = outputs[0].mean()

        loss.backward()
        optimizer.step()
        scheduler.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

print('Training complete!')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 1/1 [00:19<00:00, 19.48s/it, loss=-0.0381]

Training complete!


In [24]:
model.save_pretrained('/content/fine_tuned_bert_model')


In [42]:
!pip install rouge-score
!pip install nltk
!pip install tqdm


ERROR: Could not find a version that satisfies the requirement datatests (from versions: none)
ERROR: No matching distribution found for datatests


In [45]:
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import BertForQuestionAnswering, BertTokenizer, pipeline
import torch
import pandas as pd

from sklearn.metrics import f1_score

# Load the fine-tuned model
question_answering_model = BertForQuestionAnswering.from_pretrained('/content/fine_tuned_bert_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def predict_answer(question_answering_model, tokenizer, question, device="cuda"):
  """
  Generates an answer to a given question using a pre-trained question answering model

  Args:
      question_answering_model: The loaded question answering model.
      tokenizer: The tokenizer used to preprocess text for the model.
      question: The user-provided question.
      device: The device to use for computation ("cuda" for GPU, "cpu" otherwise).

  Returns:
      The predicted answer string.
  """

  inputs = tokenizer.encode_plus(question, return_tensors="pt").to(device)
  with torch.no_grad():
    outputs = question_answering_model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
  return answer

test = pd.read_csv('/content/test_data.csv')

def evaluate_predictions(ground_truth_answers, generated_answers):
  """
  Computes various evaluation metrics for a set of predicted answers

  Args:
      ground_truth_answers: A list of reference answer strings.
      generated_answers: A list of predicted answer strings.

  Returns:
      A tuple containing average BLEU, ROUGE-L, and F1 scores.
  """

  bleu_scores = []
  rouge_l_scores = []
  f1_scores = []

  rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

  for ref, pred in zip(ground_truth_answers, generated_answers):
    # BLEU score
    bleu_score = sentence_bleu([ref.split()], pred.split())
    bleu_scores.append(bleu_score)

    # ROUGE-L score
    rouge_l_score = rouge.score(ref, pred)["rougeL"].fmeasure
    rouge_l_scores.append(rouge_l_score)

    # F1-score
    ref_tokens = set(ref.split())
    pred_tokens = set(pred.split())
    common_tokens = ref_tokens.intersection(pred_tokens)

    if len(common_tokens) == 0:
      f1 = 0.0
    else:
      precision = len(common_tokens) / len(pred_tokens)
      recall = len(common_tokens) / len(ref_tokens)
      f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1)

  avg_bleu = sum(bleu_scores) / len(bleu_scores)
  avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
  avg_f1 = sum(f1_scores) / len(f1_scores)

  return avg_bleu, avg_rouge_l, avg_f1

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
question_answering_model.to(device)

# Assuming 'test' is a DataFrame with 'question' and 'answer' columns
questions = test.question
ground_truth_answers = test.answer

predicted_answers = [predict_answer(question_answering_model, tokenizer, q, device) for q in questions]

avg_bleu, avg_rouge_l, avg_f1 = evaluate_predictions(ground_truth_answers, predicted_answers)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average F1 Score: {avg_f1}")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 8.506863021393776e-05
Average ROUGE-L Score: 0.010821782976539216
Average F1 Score: 0.010937068334262954
